# Baseline RUL Modeling (FD001)

This notebook builds interpretable baseline models for Remaining Useful Life prediction using the FD001 subset of the NASA CMAPSS dataset.

The goal is not maximum prediction performance, but to establish:
- a leakage-free modeling pipeline
- a transparent baseline for comparison only testing linear models (No time-series models or deep learning at this stage)
- a reference point for more complex models later (No hyperparam tuning at this stage)


## Imports and global configuration for plots

In [137]:
# import data analysis and modeling libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# import modeling libraries
import statsmodels.api as sm
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import GroupKFold

# import data loader module
from src.data_loader import load_fd001
from pathlib import Path

# Random state
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [113]:
# Global plot appearance settings
mpl.rcParams.update({

    # Figure & Axes
    "figure.figsize": (10, 6),
    "figure.dpi": 120,
    "figure.facecolor": "#0b0f1a",
    "axes.facecolor": "#0b0f1a",
    "axes.edgecolor": "#8aa2c8",
    "axes.linewidth": 0.8,

    # Grid
    "axes.grid": True,
    "grid.color": "#1f2a44",
    "grid.linestyle": "--",
    "grid.linewidth": 0.6,
    "grid.alpha": 0.6,

    #  Text
    "text.color": "#e6e6eb",
    "axes.labelcolor": "#e6e6eb",
    "xtick.color": "#c7d0e0",
    "ytick.color": "#c7d0e0",
    "axes.titleweight": "bold",
    "axes.titlesize": 14,
    "axes.labelsize": 12,

    # ===== Ticks =====
    "xtick.major.size": 4,
    "ytick.major.size": 4,
    "xtick.minor.size": 2,
    "ytick.minor.size": 2,

    # Lines
    "lines.linewidth": 2.0,
    "lines.markersize": 5,

    # Legend
    "legend.facecolor": "#0b0f1a",
    "legend.edgecolor": "#8aa2c8",
    "legend.framealpha": 0.9,
    "legend.fontsize": 10,

    # Color Cycle
    "axes.prop_cycle": mpl.cycler(color=[
        "#4cc9f0",
        "#f72585",
        "#b5179e",
        "#7209b7",
        "#560bad",
        "#480ca8",
        "#3a86ff",
        "#ffd166"
    ])
})

## Load preprocessed data

In [114]:
# Establish path and load in data using data loading function
PROJECT_ROOT = Path().resolve().parents[0]
DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / 'fd001_processed.csv'
df = load_fd001(DATA_PATH)

In [115]:
# Check if load-in was successful
SENSORS = ["sensor_4", "sensor_11", "sensor_15"]
df.describe()

,unit_number,time_in_cycles,operation_setting_1,operation_setting_2,operation_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL,RUL_capped
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,...,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705,107.807862,86.829286
std,29.227633,68.880990,0.002187,0.000293,0.0,0.00,0.500053,6.131150,9.000605,1.776400e-15,...,19.076176,0.037505,1.387812e-17,1.548763,0.0,0.0,0.180746,0.108251,68.880990,41.673699
min,1.000000,1.000000,-0.008700,-0.000600,100.0,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,...,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200,0.000000,0.000000
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,...,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800,51.000000,51.000000
50%,52.000000,104.000000,0.000000,0.000000,100.0,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,...,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900,103.000000,103.000000
75%,77.000000,156.000000,0.001500,0.000300,100.0,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,...,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800,155.000000,125.000000
max,100.000000,362.000000,0.008700,0.000600,100.0,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,...,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400,361.000000,125.000000


## Engine level train/validation split

In [116]:
# Each engine should start at cycle 1
starts = df.groupby("unit_number")["time_in_cycles"].min()
assert (starts == 1).all(), "Some engines do not start at cycle 1"

# Cycles strictly increase within each engine
assert (
    df.sort_values(["unit_number", "time_in_cycles"])
      .groupby("unit_number")["time_in_cycles"]
      .apply(lambda x: x.is_monotonic_increasing)
      .all()
), "Cycle ordering issue detected"


In [117]:
# Grouping by engine id
groups = df['unit_number']

# Engine level train/validation split
gss = GroupShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=RANDOM_STATE
)

train_idx, val_idx = next(gss.split(df, groups=groups))

train_df = df.iloc[train_idx].copy()
val_df = df.iloc[val_idx].copy()

# Extract ID from each split
train_engines = set(train_df['unit_number'])
val_engines = set(val_df['unit_number'])

# Stops execution if any engine appears in both sets - preventing engine data leaks
assert train_engines.isdisjoint(val_engines), "Engine leak detected"

# Define features and target
X_train = train_df[SENSORS]
y_train = train_df['RUL_capped']

X_val = val_df[SENSORS]
y_val = val_df["RUL_capped"]

# Check that splits are properly proportioned
len(train_engines), len(val_engines)

(80, 20)

## Feature Scaling and preprocessing

In [118]:
# Initialize scaler
scaler = StandardScaler()

# Fit training data and validation data
X_train_scaled_array = scaler.fit_transform(X_train)
X_val_scaled_array = scaler.fit_transform(X_val)

# Turn array back to DF
X_train_scaled = pd.DataFrame(
    X_train_scaled_array,
    columns=SENSORS,
    index=X_train.index
)

X_val_scaled = pd.DataFrame(
    X_val_scaled_array,
    columns=SENSORS,
    index=X_val.index
)

In [119]:
# Check features are standardized
X_train_scaled.describe()

,sensor_4,sensor_11,sensor_15
count,1.656100e+04,1.656100e+04,1.656100e+04
mean,-4.397719e-15,-3.026918e-14,5.130101e-15
std,1.000030e+00,1.000030e+00,1.000030e+00
min,-2.642296e+00,-2.595427e+00,-3.042777e+00
25%,-7.249134e-01,-7.137854e-01,-7.232079e-01
50%,-1.092092e-01,-1.116601e-01,-8.865916e-02
75%,6.180355e-01,6.033638e-01,6.152100e-01
max,3.637440e+00,3.689256e+00,3.806617e+00


## Baseline model
Since RUL prediction is a regression problem, we start with a linear regression model as a baseline reference. This provides a simple, interpretable benchmark before moving to more complex models. Many of the sensors identified in EDA show approximately monotonic degradation behavior, so a linear function should capture at least the first-order relationship between sensor values and RUL. Linear regression also has high bias and low variance, which makes it well suited as a baseline which does well at establishing a performance floor that later machine learning and time-series models can improve upon.

In [120]:
# Add intercept
X_train_ols = sm.add_constant(X_train_scaled)
X_val_ols = sm.add_constant(X_val_scaled)

# Fit ols on training
ols_model = sm.OLS(y_train, X_train_ols)
ols_results = ols_model.fit()

In [121]:
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:             RUL_capped   R-squared:                       0.652
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                 1.033e+04
Date:                Sat, 17 Jan 2026   Prob (F-statistic):               0.00
Time:                        22:51:08   Log-Likelihood:                -76528.
No. Observations:               16561   AIC:                         1.531e+05
Df Residuals:                   16557   BIC:                         1.531e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         86.9588      0.191    455.180      0.0

For the sensors we can see the coefficients are all significant according to a confidence level of 95%. The coefficients are negative showing that the sensors degrade as RUL increases, with sensor_11 being the most informative - it has the biggest absolute coefficient.

In [122]:
# Predict on validation for prediction and store results for evaluation
y_val_pred_ols = ols_results.predict(X_val_ols)

In [123]:
# Evaluation function
def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

In [124]:
mae_ols, rmse_ols, r2_ols = regression_metrics(y_val, y_val_pred_ols)

In [125]:
print(f'MAE: {mae_ols} \nRMSE: {rmse_ols} \nR2: {r2_ols}')

MAE: 19.06302417791126 
RMSE: 23.395778999005753 
R2: 0.6854593208555824


MAE: On average our prediction is off by 19 cycles given the regression function
RMSE: Since the RMSE is larger than MAE that signifies that large errors are rare but still exist
R2: 68.54% of variability in y explained by the selected sensors

## Baseline model W Regularization

In [126]:
# Set alpha grid -  builds alpha values from 10^-4 to 10^4
alphas = np.logspace(-4, 4, 30)
alphas[:5], alphas[-5:]

(array([0.0001    , 0.00018874, 0.00035622, 0.00067234, 0.00126896]),
 array([  788.04628157,  1487.35210729,  2807.21620394,  5298.31690628,
        10000.        ]))

In [138]:
# Set up splitter
group_kfold = GroupKFold(n_splits=5)
groups_train = train_df['unit_number'].values

# Manual CV to avoid data leakage with prebuilt CV function
alpha_mae = []


for alpha in alphas:
    fold_maes = []

    for tr_idx, va_idx in group_kfold.split(X_train_scaled, y_train, groups_train):
        X_tr = X_train_scaled.iloc[tr_idx]
        y_tr = y_train.iloc[tr_idx]

        X_va = X_train_scaled.iloc[va_idx]
        y_va = y_train.iloc[va_idx]

        model = Ridge(alpha=alpha)
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_va)
        fold_maes.append(mean_absolute_error(y_va, y_pred))

    alpha_mae.append(np.mean(fold_maes))

In [141]:
# Choose best alpha -  regularization strength
best_alpha = alphas[np.argmin(alpha_mae)]
best_alpha

0.0001

In [143]:
# Train and fit ridge model
ridge = Ridge(alpha=best_alpha)
ridge.fit(X_train_scaled, y_train)

Ridge(alpha=0.0001)

In [144]:
# Evaluate on validation engines
y_val_pred_ridge = ridge.predict(X_val_scaled)
mae_ridge, rmse_ridge, r2_ridge = regression_metrics(y_val, y_val_pred_ridge)

In [145]:
# Print metrics
print(f'Best Alpha: {best_alpha}')
print(f'MAE: {mae_ridge}')
print(f'RMSE: {rmse_ridge}')
print(f'R2: {r2_ridge}')

Best Alpha: 0.0001
MAE: 19.06302418473698
RMSE: 23.395779001261257
R2: 0.6854593207949349


In [147]:
# Inspect coefficients
pd.Series(ridge.coef_, index=SENSORS).sort_values()

sensor_11   -15.353609
sensor_4    -11.947351
sensor_15    -8.829507
dtype: float64

Ridge regression essentially gives us the same results as OLS, alpha is very low. Regularization does little to improve metrics.